In [1]:

import sys
sys.path.insert(1, '/home/msantolaria/Documents/MyPythonLibrary/ClimAnag/')
import climbasis as climb
from climbasis import *
import domain as dom
import myplot
import glob
from eofs.xarray import Eof
from myplot import *
import obsinfo as obs
from obsinfo import *



In [2]:
source='/home/msantolaria/Documents/MyResearch/AnalysisObservations/'
resultsDir=source + 'Results/'
plotsDir=source + 'Plots/'

In [3]:
sourceData='/home/msantolaria/Documents/Data/'
sourceData1=sourceData +'noaaV2c/'
sourceData2=sourceData +'era5/'
sourceData3=sourceData +'aphro/'
sourceData4=sourceData +'gpcc/'
sourceData5=sourceData +'cru/'
sourceData6=sourceData +'gpcp/'

In [5]:
iyr=1951
fyr=2013
domain='EURA'
variable='pr'
units='mm/day'
season='DJF'
exp='obs'

In [6]:
model3='aphro'
fileName3='APHRO_mon_MA_025deg_V1101_EXR1.1951-2015.nc'
ds3 = xr.open_dataset(sourceData3+fileName3)['precip']
field3=dom.field_dom(ds3,domain)
print(ds3.attrs)

{'long_name': 'daily precipitation analysis interpolated onto 0.25deg grids [mm/day]', 'units': 'mm/day', 'cell_methods': 'time: mean', 'level_description': 'Earth surface'}


In [7]:
data4='gpcc'
model4='gpcc'
info4=obs.get_obs('precip',data4)
print(info4)
fileName4=info4.get('filename')
ds4= xr.open_dataset(sourceData+data4+'/'+fileName4)['precip']
#month_length = ds0.time.dt.days_in_month
#
#ds=ds0/month_length
units='mm'
print(ds4)
# replace all values equal to -1000 with np.nan
#ds = ds_fill[variable].where(ds_fill[variable] != -1000.)
if info4.get('gridlon')=='0_360':
    ds4=dom.shifting_grid(ds4)
lat,lon=climb.latlon(ds4)
ylat=ds4.coords[lat]
xlon=ds4.coords[lon]


field40=dom.field_dom(ds4,domain)

month_length = field40.time.dt.days_in_month
#
field4=field40/month_length

var filename units gridlon gridlat
month_length = field0dom.time.dt.days_in_month
field4=field40/month_length
{'var': 'precip', 'filename': 'precip.mon.total.gpcc.v2018.nc', 'units': 'mm', 'gridlon': '0_360', 'gridlat': '90_-90'}
<xarray.DataArray 'precip' (time: 1512, lat: 360, lon: 720)>
[391910400 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) datetime64[ns] 1891-01-01 1891-02-01 ... 2016-12-01
Attributes:
    units:         mm
    var_desc:      Precipitation
    level_desc:    Surface
    parent_stat:   Observations
    long_name:     GPCC Monthly total of precipitation
    valid_range:   [   0. 8000.]
    statistic:     Total
    level:         Surface
    actual_range:  [   0.   4552.43]
    dataset:       GPCC Precipitation 0.5degree V2018 Full Reanalysis


In [8]:
model5='cru'
fileName5='cru_ts4.05.1901.2020.pre.dat.nc'
ds5 = xr.open_dataset(sourceData5+fileName5)['pre']
print(ds5.attrs)

field50=dom.field_dom(ds5,domain)

month_length = field50.time.dt.days_in_month
#
field5=field50/month_length

{'long_name': 'precipitation', 'units': 'mm/month', 'correlation_decay_distance': 450.0}


In [9]:
ds5 

<xarray.DataArray 'pre' (time: 1440, lat: 360, lon: 720)>
[373248000 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 1901-01-16 1901-02-15 ... 2020-12-16
Attributes:
    long_name:                   precipitation
    units:                       mm/month
    correlation_decay_distance:  450.0

In [ ]:
dsList=[field3,field4,field5]
modelList=[model3,model4,model5]

In [ ]:
valsList=[]
anomsList=[]

for elem in dsList:
    if season[0]=='m':
        rmon=int(season.split('mon')[1])
        vals,anoms=climb.monthly_selection(elem,rmon,iyr,fyr)
    else:
        vals,anoms=climb.seasonal_selection(elem,season,iyr,fyr)
    valsList.append(vals)
    anomsList.append(anoms)

In [ ]:
climList=[]
stdList=[]
for elem in valsList:
    c=elem.mean('time')
    s=elem.std('time')
    climList.append(c)
    stdList.append(s)

In [ ]:
##Plotting
###
infoplot=myplot.get_var_infos(variable)
cmapClim=infoplot.get('cmap')
units=infoplot.get('units')
clevsClim= np.arange(0,5.25,0.25)#infoplot.get('levels')

cmapStd=infoplot.get('cmap_std')
clevsStd=np.arange(0,40,5)#infoplot.get('levels_std')

clevsT=np.arange(-0.5,0.55,0.05)#infoplot.get('levels_diff')
cmapT=infoplot.get('cmap_diff')

clevsRatio=np.arange(0,1.1,0.1)
cmapRatio='OrRd'

extend=infoplot.get('extend')
extentTF=False


In [ ]:
for i,clim in enumerate(climList):
##Clim
##
    latS,latN,lonW,lonE,latlim,lonlim=dom.coord_domain(domain)

    fig,axs= plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()},figsize=(8,10))
    lat,lon=climb.latlon(clim)
    lons, lats = np.meshgrid(clim[lon] ,clim[lat])
    CS1=axs.contourf(lons,lats, clim,clevsClim,
            transform=ccrs.PlateCarree(),
            cmap=cmapClim,extent='both')
# Draw the coastines for each subplot
    axs.coastlines()
    axs.add_feature(cfeature.BORDERS, linestyle=':', alpha=1)
#axs.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '50m', edgecolor='None', facecolor='None'))
    if extentTF==True:
        axs.set_extent([lonW, lonE, latS,latN])
    # Longitude and latitude labels
# Adjust the location of the subplots on the page to make room for the colorbar
    fig.subplots_adjust(bottom=0.35, top=0.7, left=0.20, right=0.80,
                wspace=0.05, hspace=0.5)
# Add a colorbar axis at the bottom of the graph
#([xmin,ymin,dx,dy])
    cbar_ax = fig.add_axes([0.2, 0.35, 0.6, 0.02])
# Draw the colorbar
    cbar=fig.colorbar(CS1, cax=cbar_ax,orientation='horizontal',label='%s'%(units))
## Add a big title at the top
    ofileC='clim_'+variable+'_'+modelList[i]+'_'+exp+'_'+domain+'_'+season+'_'+str(iyr)+'_'+str(fyr)
    plt.suptitle(ofileC,y=0.68)
    fig.savefig(plotsDir+ofileC+'.png',format='png')
    print('Figure save at ',plotsDir, 'as',ofileC)
    plt.show()


In [ ]:
#climList[1].plot()

In [ ]:
parList=[]
trendList=[]
interceptList=[]
rvalueList=[]
pvalueList=[]
stderrList=[]
#---------------------------------------------------------
for elem in anomsList:
    par=climb.trend_vect(elem.time,elem,'time')
    parList.append(par)
    trendList.append(par[0])
    interceptList.append(par[1])
    rvalueList.append(par[2])
    pvalueList.append(par[3])
    stderrList.append(par[4])


In [ ]:
for i,par in enumerate(parList):
##
##Spatial trend
###
    fig,axs= plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()},figsize=(8,10))

    lat,lon=climb.latlon(par[0])
    lons, lats = np.meshgrid(par[0][lon] ,par[0][lat])
    CS1=axs.contourf(lons,lats, 10*par[0][:,:],clevsT,
                transform=ccrs.PlateCarree(),
                cmap=cmapT,extent='both')
    levels=[0,0.1,1.0]
    cs = axs.contourf(lons,lats,par[3][:,:], transform=ccrs.PlateCarree(),levels=levels,
                hatches=["+", ""], alpha=0.)

# Draw the coastines for each subplot
    axs.coastlines()
    axs.add_feature(cfeature.BORDERS, linestyle=':', alpha=1)
#axs.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '50m', edgecolor='None', facecolor='None'))
    if extentTF==True:
        axs.set_extent([lonW, lonE, latS,latN])
    # Longitude and latitude labels
# Adjust the location of the subplots on the page to make room for the colorbar
    fig.subplots_adjust(bottom=0.35, top=0.7, left=0.20, right=0.80,
                wspace=0.05, hspace=0.5)
# Add a colorbar axis at the bottom of the graph
#([xmin,ymin,dx,dy])
    cbar_ax = fig.add_axes([0.2, 0.35, 0.6, 0.02])
# Draw the colorbar
    cbar=fig.colorbar(CS1, cax=cbar_ax,orientation='horizontal',label='%s/dec'%(units))
## Add a big title at the top
    ofileT='spatialtrend_'+variable+'_'+modelList[i]+'_'+exp+'_'+domain+'_'+season+'_'+str(iyr)+'_'+str(fyr)
    plt.suptitle(ofileT,y=0.68)
    fig.savefig(plotsDir+ofileT+'.png',format='png')
    print('Figure save at ',plotsDir, 'as',ofileT)
    plt.show()


In [ ]:
##Computing time series of seasonal spatial average ----------------------------
print('Computing time series of seasonal spatial average')
ts_seasonList=[]
for i,anoms in enumerate(anomsList):
    ts_season=climb.spatial_average(anoms)
    ts_seasonList.append(ts_season)
    plotnameTs='timeseries_anoms_'+variable+'_'+modelList[i]+'_'+exp+'_'+domain+'_'+season+'_'+str(iyr)+'_'+str(fyr)
    np.savetxt(resultsDir+plotnameTs+'.txt',ts_season)
#print('saving .txt at',resultsDir+plotnameTs)


In [ ]:
xd=np.arange(0,len(ts_season),1)

fig, ax = plt.subplots(figsize=(12, 8))
my_ticks=xd+iyr
print(xd,my_ticks)
ticks=np.arange(iyr,fyr+1,1)
#ax.plot(xd,ts_season,color='b', linewidth=2)
ax.plot(xd, ts_seasonList[0], 'b:',label=modelList[0])
ax.plot(xd, ts_seasonList[1], 'r:',label=modelList[1])
ax.plot(xd, ts_seasonList[2], 'g:',label=modelList[2])
#ax.plot(xd, parmed[1] + parmed[0] * xd, 'r-',label="%s (%.2f)" % ('Theil-Sen', 10*parmed[0]))
#ax.plot(xd, parmed[1] + parmed[2] * xd, 'r--')
#ax.plot(xd, parmed[1] + parmed[3] * xd, 'r--')
#ax.plot(xd, par[1] + par[0] * xd, 'b-',label="%s (%.2f);%s (%.2f)" % ('Lin reg', 10*par[0],'p-val',par[3]))
#ax.plot(xd, parmk[8] + parmk[7] * xd,'p-',label="%s (%.2f) sig %s" % ('MK',i 10*parmk[7],parmk[0]))

#ax.axhline(0, color='k')
#ax.set_ylim(0, 3)
ax.set_xlabel('Year')
ax.set_ylabel('%s'%(units))
ax.set_title(plotnameTs)
freq=5
plt.xticks(xd,my_ticks, rotation='vertical')
plt.legend()
#plt.savefig(plotsDir+plotnameTs+'.png',format='png')
plt.show()
